# Writing kids jokes with LSTM

The great/frustrating thing about this is that it doesn't need to work all that well in order to be satisfactory.

The inspiration for this is one of my favorite twitter accounts, found [here](https://twitter.com/KidsWriteJokes)

The data prep and model code was cobbled together mostly from these two sources:

http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/  
https://beneyal.github.io/data-science-rnn/

In [177]:
import json
import pytumblr
import tweepy
from tweepy import OAuthHandler
from tweepy import AppAuthHandler
from tweepy import Stream
import urllib2
import time
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

# Collect tumblr data

For all data collection api token information is masked

In [31]:
client = pytumblr.TumblrRestClient(
  'XXX',
  'XXX',
  'XXX',
  'XXX'
)

In [32]:
posts = client.posts('badkidsjokes')
total_posts = range(posts['total_posts'])
for i in total_posts[20::20]:
    page = client.posts('badkidsjokes', offset=i)
    posts['posts'].extend(page['posts'])

In [33]:
with open('tumblrdat.json', 'w') as f:
    json.dump(posts, f)

# twitter data

In [34]:
access_token = "XXX"
access_token_secret = "XXX"
consumer_key = "XXX"
consumer_secret = "XXX"


auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [149]:
alltweets = []
new_tweets = api.user_timeline(screen_name='KidsWriteJokes', count=200)
alltweets.extend(new_tweets)
if len(new_tweets)>0:
    oldest = alltweets[-1].id - 1
    while len(new_tweets) > 0:
        new_tweets=api.user_timeline(screen_name='KidsWriteJokes', count=200, max_id=oldest)
        alltweets.extend(new_tweets)
        oldest = alltweets[-1].id-1

In [150]:
new_tweets = api.user_timeline(screen_name='reallybadlaughs', count=200)
alltweets.extend(new_tweets)
if len(new_tweets)>0:
    oldest = alltweets[-1].id - 1
    while len(new_tweets) > 0:
        new_tweets=api.user_timeline(screen_name='reallybadlaughs', count=200, max_id=oldest)
        alltweets.extend(new_tweets)
        oldest = alltweets[-1].id-1

In [151]:
len(alltweets)

679

In [152]:
with open('twitterdat.json', 'w') as f:
    json.dump([tweet._json for tweet in alltweets], f)

# FB data

In [37]:
def request_until_succeed(url):
    """
    handles potential errors when sending a request
    """
    
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)

    return response.read()

In [38]:
app_id = 'XXX'
app_secret = 'XXX'
baseurl = 'https://graph.facebook.com/v2.8/kidswritejokes/feed/?fields=message'
access_token = '&access_token=' + app_id + '|' + app_secret
url = baseurl+access_token
dat = json.loads(request_until_succeed(url))
items=dat

In [39]:
has_next_page = True
while has_next_page:
    if 'paging' not in dat.keys():
        break
    
    dat = json.loads(request_until_succeed(dat['paging']['next']))
    items['data'].extend(dat['data'])

In [41]:
with open('facebookdat.json', 'w') as f:
    json.dump(items, f)

# Open and combine data

In [156]:
with open('facebookdat.json') as f:
    fb_data = json.load(f)

In [157]:
with open('twitterdat.json') as f:
    twitter_data = json.load(f)

In [158]:
with open('tumblrdat.json') as f:
    tumblr_data = json.load(f)

In [159]:
jokes = []
for i in fb_data['data']:
    jokes.append(i['message'])

In [160]:
for i in tumblr_data['posts']:
    jokes.append(i['body'])

In [161]:
for i in twitter_data:
    jokes.append(i['text'])

In [55]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.optimizers import RMSprop
from bs4 import BeautifulSoup

Using Theano backend.


In [180]:
jokes = [BeautifulSoup(j, 'lxml').get_text() for j in jokes]
#[j.lower() for j in jokes]

# Prep the data for modeling

In [181]:
CORPUS_LENGTH = None

def get_corpus(joke_dat, verbose=0):
    emoji_pattern = re.compile(
        u"(\ud83d[\ude00-\ude4f])|"  # emoticons
        u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
        u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
        u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
        u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
        u'\U0001F300-\U0001F64F'
        u'\U0001F680-\U0001F6FF'
        u'\u2600-\u26FF\u2700-\u27BF]+', 
        flags=re.UNICODE)
    joke_dat = [t for t in joke_dat if 'http' not in t]
    joke_dat = [t for t in joke_dat if '@' not in t]
    joke_dat = [emoji_pattern.sub(r'', t) for t in joke_dat]
    corpus = u' '.join(joke_dat)
    global CORPUS_LENGTH
    CORPUS_LENGTH = len(corpus)
    if verbose:
        print('Corpus Length:', CORPUS_LENGTH)
    return corpus

corpus = get_corpus(jokes, verbose=1)

('Corpus Length:', 123297)


In [182]:
N_CHARS = None

def create_index_char_map(corpus, verbose=0):
    chars = sorted(list(set(corpus)))
    global N_CHARS
    N_CHARS = len(chars)
    if verbose:
        print('No. of unique characters:', N_CHARS)
    char_to_idx = {c: i for i, c in enumerate(chars)}
    idx_to_char = {i: c for i, c in enumerate(chars)}
    return chars, char_to_idx, idx_to_char

chars, char_to_idx, idx_to_char = create_index_char_map(corpus, verbose=1)

('No. of unique characters:', 93)


In [183]:
MAX_SEQ_LENGTH = 40
SEQ_STEP = 3
N_SEQS = None

def create_sequences(corpus, verbose=0):
    sequences, next_chars = [], []
    for i in range(0, CORPUS_LENGTH - MAX_SEQ_LENGTH, SEQ_STEP):
        sequences.append(corpus[i:i + MAX_SEQ_LENGTH])
        next_chars.append(corpus[i + MAX_SEQ_LENGTH])
    global N_SEQS
    N_SEQS = len(sequences)
    if verbose:
        print('No. of sequences:', len(sequences))
    return np.array(sequences), np.array(next_chars)

sequences, next_chars = create_sequences(corpus, verbose=1)

('No. of sequences:', 41086)


In [184]:
def one_hot_encode(sequences, next_chars, char_to_idx):
    X = np.zeros((N_SEQS, MAX_SEQ_LENGTH, N_CHARS), dtype=np.bool)
    y = np.zeros((N_SEQS, N_CHARS), dtype=np.bool)
    for i, sequence in enumerate(sequences):
        for t, char in enumerate(sequence):
            X[i, t, char_to_idx[char]] = 1
            y[i, char_to_idx[next_chars[i]]] = 1
    return X, y

X, y = one_hot_encode(sequences, next_chars, char_to_idx)

In [185]:
def build_model(hidden_layer_size=128, dropout=0.2, learning_rate=0.01, verbose=0):
    model = Sequential()
    model.add(LSTM(hidden_layer_size, return_sequences=True, input_shape=(MAX_SEQ_LENGTH, N_CHARS)))
    model.add(Dropout(dropout))
    model.add(LSTM(hidden_layer_size, return_sequences=False))
    model.add(Dropout(dropout))
    model.add(Dense(N_CHARS, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=learning_rate))
    if verbose:
        print('Model Summary:')
        model.summary()
    return model

model = build_model(verbose=1)

Model Summary:
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_5 (LSTM)                    (None, 40, 128)       113664      lstm_input_3[0][0]               
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 40, 128)       0           lstm_5[0][0]                     
____________________________________________________________________________________________________
lstm_6 (LSTM)                    (None, 128)           131584      dropout_5[0][0]                  
____________________________________________________________________________________________________
dropout_6 (Dropout)              (None, 128)           0           lstm_6[0][0]                     
____________________________________________________________________________

In [188]:

def train_model(model, X, y, batch_size=128, nb_epoch=60, verbose=0):
    checkpointer = ModelCheckpoint(filepath="weights_seq40_step3.hdf5", monitor='loss', verbose=verbose, save_best_only=True, mode='min')
    model.fit(X, y, batch_size=batch_size, nb_epoch=nb_epoch, verbose=verbose, callbacks=[checkpointer])

train_model(model, X, y, verbose=1)

Epoch 1/60
41086/41086 [==============================] - 126s - loss: 1.9989   
Epoch 2/60
41086/41086 [==============================] - 126s - loss: 1.8865   
Epoch 3/60
41086/41086 [==============================] - 125s - loss: 1.8074   
Epoch 4/60
41086/41086 [==============================] - 126s - loss: 1.7365   
Epoch 5/60
41086/41086 [==============================] - 127s - loss: 1.6884   
Epoch 6/60
41086/41086 [==============================] - 127s - loss: 1.6430   
Epoch 7/60
41086/41086 [==============================] - 127s - loss: 1.6082   
Epoch 8/60
41086/41086 [==============================] - 127s - loss: 1.5735   
Epoch 9/60
41086/41086 [==============================] - 128s - loss: 1.5425   
Epoch 10/60
41086/41086 [==============================] - 127s - loss: 1.5191   
Epoch 11/60
41086/41086 [==============================] - 128s - loss: 1.4950   
Epoch 12/60
41086/41086 [==============================] - 128s - loss: 1.4754   
Epoch 13/60
41086/41086 [

In [190]:
np.random.seed(1337)
def sample(preds):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 0.2
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def generate_tweets(model, corpus, char_to_idx, idx_to_char, n_tweets=10, verbose=0): 
    model.load_weights('weights_seq40_step3.hdf5')
    tweets = []
    #spaces_in_corpus = np.array([idx for idx in range(CORPUS_LENGTH) if corpus[idx] == ' '])
    for i in range(1, n_tweets + 1):
        sequence = np.random.choice(jokes)[0:MAX_SEQ_LENGTH]
        #begin = np.random.choice(spaces_in_corpus)
        tweet = u''
        #sequence = corpus[begin:begin + MAX_SEQ_LENGTH]
        tweet += sequence
        if verbose:
            print('Tweet no. %03d' % i)
            print('=' * 13)
            print('Generating with seed:')
            print(sequence)
            print('_' * len(sequence))
        for _ in range(100):
            x = np.zeros((1, MAX_SEQ_LENGTH, N_CHARS))
            for t, char in enumerate(sequence):
                x[0, t, char_to_idx[char]] = 1.0

            preds = model.predict(x, verbose=0)[0]
            next_idx = sample(preds)
            next_char = idx_to_char[next_idx]

            tweet += next_char
            sequence = sequence[1:] + next_char
        if verbose:
            print(tweet)
            print()
        tweets.append(tweet)
    return tweets

tweets = generate_tweets(model, corpus, char_to_idx, idx_to_char, verbose=1)


Exception RuntimeError: RuntimeError('Failed to retrieve old handler',) in 'h5py._errors.set_error_handler' ignored
Exception RuntimeError: RuntimeError('Failed to retrieve old handler',) in 'h5py._errors.set_error_handler' ignored


Tweet no. 001
Generating with seed:
what did the women do in bed when she he
________________________________________
what did the women do in bed when she hear and a banana with no eyes

because it was a boy and talk and said to get the sents a boy and a bo
()
Tweet no. 002
Generating with seed:
why did the zebra wear stripy pijamas
be
________________________________________
why did the zebra wear stripy pijamas
because they don’t have a fish with no eye


a cheese was a boy and has a boy and the cow and said to 
()
Tweet no. 003
Generating with seed:
Why did they let the turkey join the ban
________________________________________
Why did they let the turkey join the banana fart and a boy with a man and a boy and talk a baby poo in a banana and a boy and a boy and a pi
()
Tweet no. 004
Generating with seed:
what do cavemen eat

caves
__________________________
what do cavemen eat

cavesIYYYIIYYYYYYSYYYYYYYYYIYYYIYIIYSYYIYYYYYYIYYYYYYYIYYYYYIYYYYYYYYYYYYYYYYYYYYYYYYYIYYYYIYYYYYYYYYYY

In [191]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(sequences)
Xval = vectorizer.transform(tweets)
print(pairwise_distances(Xval, Y=tfidf, metric='cosine').min(axis=1).mean())

0.422718712303


# Summary
It does okay I suppose. It seems to get hung up on specific ideas (cows, bananas). I've toyed around with changing the sequence length (i.e. how many characters the model gets to predict the next value), and the sequence step. The sequence step is just a pragmatic alteration. Instead of modeling every possible sequence of letters, we're skipping some. This is just so I can fit this on my machine in a reasonable amount of time.

Some other modifications I've got planned are to change the way the input data is structured. I'd like to train it joke-by-joke, rather than on the entire corpus as one long document. I'll also try to change the structure of the model a little bit (more/less regularization; change the hidden layer size). But probably the most likely way of improving this thing is just to get some more data. I'm increasingly thinking that if I train it on regular jokes that the output will more closely approximate the kids jokes. As it is, the kids jokes are nonsensical enough that the bot is just amping up the nonsense.